<a href="https://colab.research.google.com/github/vinayp1120/legendary-potato/blob/main/yoloweapondetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.4/779.4 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
# prompt: unzip the zipped file with mentkoned path

!unzip /content/drive/MyDrive/weapon-detection.v2i.yolov9.zip -d /content


Streaming output truncated to the last 5000 lines.
 extracting: /content/train/labels/knife_1169_jpg.rf.a35b7eaf0e9e1589e61acc9f6f73499d.txt  
 extracting: /content/train/labels/knife_1169_jpg.rf.aa6f2aa42965fbd43637f579b570a8bd.txt  
 extracting: /content/train/labels/knife_1169_jpg.rf.c2493692b9f909255a33ee607b4a363e.txt  
 extracting: /content/train/labels/knife_1172_jpg.rf.815d43321d05bded1f7a098535b73a62.txt  
 extracting: /content/train/labels/knife_1172_jpg.rf.90ea600ae93da771ac587fe71bcf6812.txt  
 extracting: /content/train/labels/knife_1172_jpg.rf.9b41f12df28858dfca970e1257931982.txt  
 extracting: /content/train/labels/knife_1172_jpg.rf.be38c629ddbcc8bc08f3c89bf9fe9813.txt  
 extracting: /content/train/labels/knife_1172_jpg.rf.ccb65b4fbdca2ac2a65b2d04b5479835.txt  
 extracting: /content/train/labels/knife_1172_jpg.rf.d8f5eafcc3f59cf16228a903c0a1db31.txt  
 extracting: /content/train/labels/knife_1177_jpg.rf.059501ae509e75cd210d3eed354d2e04.txt  
 extracting: /content/train/l

In [8]:
import os
import random
import shutil

def create_validation_folder(train_folder_path, val_split=0.1):
    """
    This function creates a validation folder from the training folder
    with a random split based on the provided validation split ratio.

    Args:
        train_folder_path: Path to the main training folder containing images and potentially label files.
        val_split (optional): Percentage of training data to allocate for validation (default: 0.1).
    """

    if not os.path.exists(train_folder_path):
        raise ValueError(f"Training folder '{train_folder_path}' does not exist.")

    train_files = []

    # Collect all files from the training folder and its subdirectories
    for root, _, files in os.walk(train_folder_path):
        for file in files:
            train_files.append(os.path.join(root, file))

    random.shuffle(train_files)

    num_files = len(train_files)
    num_val_files = int(num_files * val_split)

    val_folder_path = os.path.join(train_folder_path, "validation")
    if not os.path.exists(val_folder_path):
        os.makedirs(val_folder_path)

    for i in range(num_val_files):
        file_path = train_files[i]
        file_name = os.path.basename(file_path)

        # Move image file
        try:
            shutil.move(file_path, os.path.join(val_folder_path, file_name))
        except FileNotFoundError:
            print(f"Warning: File '{file_name}' not found. Skipping.")
        except Exception as e:
            print(f"Error moving file '{file_name}': {e}")

# Example usage:
train_folder_path = "/content/train"  # Replace with the actual path
create_validation_folder(train_folder_path)


In [9]:
import os
import shutil

def organize_validation_folder(validation_folder_path):
    """
    Organizes the validation folder by separating images and label files into
    their respective 'images' and 'labels' subfolders.

    Args:
        validation_folder_path: Path to the validation folder containing mixed images and label files.
    """
    # Define paths for images and labels subfolders
    images_folder_path = os.path.join(validation_folder_path, 'images')
    labels_folder_path = os.path.join(validation_folder_path, 'labels')

    # Create the images and labels folders if they don't exist
    os.makedirs(images_folder_path, exist_ok=True)
    os.makedirs(labels_folder_path, exist_ok=True)

    # Get all files in the validation folder
    for file_name in os.listdir(validation_folder_path):
        file_path = os.path.join(validation_folder_path, file_name)

        # Skip directories (images and labels folders)
        if os.path.isdir(file_path):
            continue

        # Check the file extension and move accordingly
        if file_name.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.gif')):
            dest_path = os.path.join(images_folder_path, file_name)
        elif file_name.endswith('.txt'):
            dest_path = os.path.join(labels_folder_path, file_name)
        else:
            print(f"Skipping unrecognized file type: {file_name}")
            continue

        try:
            shutil.move(file_path, dest_path)
        except Exception as e:
            print(f"Error moving file '{file_name}': {e}")

# Example usage:
validation_folder_path = "/content/validation"  # Replace with the actual path
organize_validation_folder(validation_folder_path)


In [10]:
from ultralytics import YOLO
model= YOLO("yolov9c.pt")
#train on custom dataset
results = model.train(data='/content/data.yaml', epochs=10, imgsz=480)

Ultralytics YOLOv8.2.26 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=/content/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /content/train/labels... 13807 images, 3327 backgrounds, 0 corrupt: 100%|██████████| 17130/17130 [00:08<00:00, 2004.04it/s]


train: New cache created: /content/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/valid/labels... 204 images, 1873 backgrounds, 0 corrupt: 100%|██████████| 2077/2077 [00:01<00:00, 1731.82it/s]


val: New cache created: /content/valid/labels.cache
Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 154 weight(decay=0.0), 161 weight(decay=0.0005), 160 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 480 train, 480 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      10/10      6.94G     0.9892      1.142      1.434         11        480: 100%|██████████| 1071/1071 [09:29<00:00,  1.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 65/65 [00:31<00:00,  2.07it/s]

                   all       2077        251     0.0876      0.542      0.084     0.0541



10 epochs completed in 1.698 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 51.6MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 51.6MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.26 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv9c summary (fused): 384 layers, 25322332 parameters, 0 gradients, 102.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 65/65 [00:29<00:00,  2.20it/s]


                   all       2077        251     0.0877      0.542     0.0843     0.0544
               Handgun         93        118     0.0831      0.551      0.073     0.0556
                 Knife         33         33     0.0918      0.455     0.0724     0.0305
                 Rifle         78        100     0.0881       0.62      0.108     0.0772
Speed: 0.2ms preprocess, 8.8ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/train2


In [15]:
model.save("mysavedmodel.pt")

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')  # Replace with your mount point if different

# Get the path to your Google Drive folder (adjust as needed)
drive_path = "/content/gdrive/MyDrive"
model.save(f"{drive_path}/mysavedmodel.pt")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968